In [1]:
from __future__ import division
import math
import pandas as pd
import numpy as np
import matplotlib as plt
from pyveplot import *
import networkx as nx
import random
from IPython.display import SVG
%matplotlib inline

In [2]:
apgi_1953 = pd.read_table("APGI1953_Tumor_lumpy.tsv")
apgi_1955 = pd.read_table("APGI1955_Tumor_lumpy.tsv")
apgi_2049 = pd.read_table("APGI2049_Tumor_lumpy.tsv")

### Only intra-chromosomal events happening on those datasets

In [3]:
# i.e:
apgi_1953[apgi_1953["chrom"] == "16"]

,sample,chrom,sv,counts
0,APGI1953_Tumor,16,DEL,3
24,APGI1953_Tumor,16,complex,7
38,APGI1953_Tumor,16,DUP,3


In [4]:
def groupby_chrom_sv(df):
    df["counts"].astype(np.int)
    by_chrom_sv = df.groupby(["chrom", "sv"])
    #by_chrom_1953.groups
    return by_chrom_sv

## Plot a hiveplot given a pandas dataframe

In [5]:
def hiveplot(fname, dataframe):

    # a network
    g = nx.Graph()

    # our hiveplot object
    h = Hiveplot('{}.svg'.format(fname))

                  # start      end
    axis0 = Axis((200,200), (200,100), stroke="black")
    axis1 = Axis((200,200), (100,300), stroke="black")

    h.axes = [ axis0, axis1 ]
    event_color = {'DEL': 'red',
                   'INV': 'yellow',
                   'DUP': 'blue',
                   'complex': 'purple'}

    for k, v in dataframe.groups.iteritems():
        g.add_node(k[0])
        # XXX: Same indexes since it is intra-chromosomal, needs to be generalized
        g.add_edge(k[0], k[0], event=k[1], count=v[0])

        
    offset = 0
    
    for n in g.nodes():
        node = Node(n)
        try:
            offset = math.fmod(float(n), 8.0)
            offset = offset/10
        except:
            pass
        
        axis0.add_node(node, offset)
        axis1.add_node(node, offset)

    for e in g.edges():
        edge_data = g.get_edge_data(*e)
        
        h.connect(axis1, e[1], 45, 
                  axis0, e[0], -45, 
                  stroke_width=edge_data['count']/10, stroke_opacity="0.4", stroke=event_color[edge_data['event']])

    h.save()

## Plot the network representation first

In [ ]:
nx.draw_networkx(g)

In [6]:
hiveplot("apgi_1953", groupby_chrom_sv(apgi_1953))
hiveplot("apgi_1955", groupby_chrom_sv(apgi_1955))
hiveplot("apgi_2049", groupby_chrom_sv(apgi_1955))